In [ ]:
Estudo parametrico de um algoritmo imunológico em um problema de classificação

In [1]:
import numpy as np
from numpy import random
import clonalg
import pandas as pd
from sklearn import preprocessing

Anticorpos treinados com exemplos de iris virginica [label 2]

In [2]:
df = pd.read_csv('dataset/iris.csv', header=None)

df[4] = pd.Categorical(df[4]).codes
labels = df[[4]].copy()

df = df.drop(columns=[4])

#Standard Scaler
scaler = preprocessing.StandardScaler()
train = pd.DataFrame(scaler.fit_transform(df.values), columns=df.columns, index=df.index)

#Train dataset
train = train.join(labels)
train = train.loc[train[4] == 2]
train = train.drop(columns=[4])

StandartScaler é o método de normalização que apresenta os melhores resultados

In [3]:
feature_num = len(train.axes[1])
feature_max = train.max()
feature_max  = feature_max.max()
feature_min = train.min()
feature_min  = feature_min.min()

Definição de hiperparâmetros

In [4]:
# Threshold for antibody-antigen reaction
threshold = 0.1

#Hyperparameters
population_size = 100
selection_size = 10

clone_rate = 5
mutation_rate = 0.2
stop_codition = 10000

random_cells_num = 30

In [5]:
# Threshold for antibody-antigen reaction
threshold = 0.1

#Hyperparameters
population_size = 1000
selection_size = 10
memory_set_percentage = 1
antibody_antigen = 0.1

clone_rate = 5
mutation_rate = 0.2
stop_codition = 10000

random_cells_num = 30

Conversão de dataframe para numpy array

In [6]:
train = train.to_numpy()

Seleção clonal de melhores anticorpos

In [7]:
len(train)

50

Estrutura da célula (cell) = [0] = genes | [1] = afinidade

![](resources/V7zasui.png)

In [10]:
best = 0.0001
valor = 0.01
print(clonalg.float_into_distance(valor, best))

100.0


In [23]:
stop = 0
population = clonalg.create_random_cells(population_size, feature_num, feature_min, feature_max)

while stop != stop_codition:
    # 1.
    for antigen in train:
        # 1.1
        population_affinity = [(cell, clonalg.affinity(cell, antigen)) for cell in population]
        # 1.2
        population_affinity = sorted(population_affinity, key=lambda x: abs(x[1]))
        best_affinity = population_affinity[:selection_size]
        # 1.3
        clone_population = []
        for cell in best_affinity:
            cell_clones = clonalg.clone(cell, clone_rate)
            clone_population += cell_clones
        # 1.4 - 1.5
        mutaded_clone_population = []
        for cell in clone_population:
            mutated_clone = clonalg.hypermutate_variability(cell, mutation_rate, antigen)
            mutaded_clone_population.append(mutated_clone)
        # 1.6
        mutaded_clone_population.sort(key=lambda x: x[1])
        pop_size = round(len(clone_population)/100)*memory_set_percentage

        mutaded_clone_population = mutaded_clone_population[0:pop_size]
        print(antigen)


    break


[0.55333328 0.56925129 1.27454998 1.71090158]
[-0.05250608 -0.8191665   0.76275864  0.92206376]
[ 1.52267624 -0.1249576   1.21768427  1.1850097 ]
[ 0.55333328 -0.35636057  1.04708716  0.79059079]
[ 0.79566902 -0.1249576   1.16081857  1.31648267]
[ 2.12851559 -0.1249576   1.6157442   1.1850097 ]
[-1.14301691 -1.28197243  0.42156442  0.65911782]
[ 1.76501198 -0.35636057  1.44514709  0.79059079]
[ 1.03800476 -1.28197243  1.16081857  0.79059079]
[1.64384411 1.26346019 1.33141568 1.71090158]
[0.79566902 0.33784833 0.76275864 1.05353673]
[ 0.67450115 -0.8191665   0.87649005  0.92206376]
[ 1.15917263 -0.1249576   0.99022146  1.1850097 ]
[-0.17367395 -1.28197243  0.70589294  1.05353673]
[-0.05250608 -0.58776353  0.76275864  1.57942861]
[0.67450115 0.33784833 0.87649005 1.44795564]
[ 0.79566902 -0.1249576   0.99022146  0.79059079]
[2.24968346 1.72626612 1.67260991 1.31648267]
[ 2.24968346 -1.05056946  1.78634131  1.44795564]
[ 0.18982966 -1.97618132  0.70589294  0.39617188]
[1.2803405  0.337848

In [24]:
clone_population

[(array([ 0.56794719,  0.9469279 ,  0.37298593, -0.38796788]),
  0.00283951418886641),
 (array([ 0.56794719,  0.9469279 ,  0.37298593, -0.38796788]),
  0.00283951418886641),
 (array([ 0.56794719,  0.9469279 ,  0.37298593, -0.38796788]),
  0.00283951418886641),
 (array([ 0.56794719,  0.9469279 ,  0.37298593, -0.38796788]),
  0.00283951418886641),
 (array([ 0.56794719,  0.9469279 ,  0.37298593, -0.38796788]),
  0.00283951418886641),
 (array([ 0.56794719,  0.9469279 ,  0.37298593, -0.38796788]),
  0.00283951418886641),
 (array([ 0.56794719,  0.9469279 ,  0.37298593, -0.38796788]),
  0.00283951418886641),
 (array([ 0.56794719,  0.9469279 ,  0.37298593, -0.38796788]),
  0.00283951418886641),
 (array([ 0.56794719,  0.9469279 ,  0.37298593, -0.38796788]),
  0.00283951418886641),
 (array([ 0.56794719,  0.9469279 ,  0.37298593, -0.38796788]),
  0.00283951418886641),
 (array([ 0.56794719,  0.9469279 ,  0.37298593, -0.38796788]),
  0.00283951418886641),
 (array([ 0.56794719,  0.9469279 ,  0.37298

In [25]:
mutaded_clone_population

[(array([ 0.56796503,  0.94697814,  0.37309443, -0.38796646]),
  0.0030175214555647756),
 (array([ 0.56810316,  0.94693377,  0.37302365, -0.38793605]),
  0.0030709150957417553),
 (array([ 0.56811695,  0.9469837 ,  0.37300295, -0.38796495]),
  0.0030850176135914786),
 (array([ 0.56811775,  0.94694988,  0.37301636, -0.38793035]),
  0.0031000047509945894),
 (array([ 0.56807895,  0.94698497,  0.37303486, -0.38793   ]),
  0.00311514531296897),
 (array([ 0.56804735,  0.94694489,  0.37313865, -0.38794837]),
  0.003128884103521301),
 (array([ 0.56796999,  0.94694286,  0.3732422 , -0.38796183]),
  0.003139589886685723),
 (array([ 0.56798877,  0.94693   ,  0.37316547, -0.38788584]),
  0.0031447717946360143),
 (array([ 0.56803882,  0.94694623,  0.37301443, -0.38779871]),
  0.0031471383905046224),
 (array([ 0.56802028,  0.94696228,  0.37307012, -0.38784158]),
  0.0031574668134497497),
 (array([ 0.56795587,  0.94710519,  0.37305762, -0.38789829]),
  0.003166766313449809),
 (array([ 0.56806448,  0.9

In [ ]:
"clone_pop = []
for i in clone_population:
    i = list(i)
    i[1] = i[1]*1000
    clone_pop.append(i)
clone_pop


In [117]:
stop = 0
population = clonalg.create_random_cells(population_size, feature_num, feature_min, feature_max)

while stop != stop_codition:
    # 1.
    for antigen in train:
        # 1.1
        population_affinity = [(cell, clonalg.affinity(cell, antigen)) for cell in population]
        # 1.2
        population_affinity = sorted(population_affinity, key=lambda x: abs(x[1]))
        best_affinity = population_affinity[:selection_size]
        # 1.3
        clone_population = []
        for cell in best_affinity:
            cell_clones = clonalg.clone(cell, clone_rate)
            clone_population += cell_clones
        # 1.4 - 1.5
        mutaded_clone_population = []
        for cell in clone_population:
            mutated_clone = clonalg.hypermutate_variability(cell, mutation_rate, antigen)
            mutaded_clone_population.append(mutated_clone)
        # 1.6
        mutaded_clone_population.sort(key=lambda x: x[1])
        pop_size = round(len(clone_population)/100)*memory_set_percentage

        mutaded_clone_population = clone_population[0:pop_size]


    break


In [118]:
clone_pop = []
for i in clone_population:
    i = list(i)
    i[1] = i[1]*1000
    clone_pop.append(i)
clone_pop


[[array([-0.41454838,  1.23821621, -0.64185303,  1.31793019]),
  2.691358374247832],
 [array([-0.41454838,  1.23821621, -0.64185303,  1.31793019]),
  2.691358374247832],
 [array([-0.41454838,  1.23821621, -0.64185303,  1.31793019]),
  2.691358374247832],
 [array([-0.41454838,  1.23821621, -0.64185303,  1.31793019]),
  2.691358374247832],
 [array([-0.41454838,  1.23821621, -0.64185303,  1.31793019]),
  2.691358374247832],
 [array([-0.41454838,  1.23821621, -0.64185303,  1.31793019]),
  2.691358374247832],
 [array([-0.41454838,  1.23821621, -0.64185303,  1.31793019]),
  2.691358374247832],
 [array([-0.41454838,  1.23821621, -0.64185303,  1.31793019]),
  2.691358374247832],
 [array([-0.41454838,  1.23821621, -0.64185303,  1.31793019]),
  2.691358374247832],
 [array([-0.41454838,  1.23821621, -0.64185303,  1.31793019]),
  2.691358374247832],
 [array([-0.41454838,  1.23821621, -0.64185303,  1.31793019]),
  2.691358374247832],
 [array([-0.41454838,  1.23821621, -0.64185303,  1.31793019]),
  

In [119]:
stop = 0
population = clonalg.create_random_cells(population_size, feature_num, feature_min, feature_max)
best_affinity_it = []

while stop != stop_codition:
    antigen = train[np.random.randint(0, len(train))]
    population_affinity = [(cell, clonalg.affinity(cell, antigen)) for cell in population]
    population_affinity = sorted(population_affinity, key=lambda x: abs(x[1]))

    # Número de melhores individuos delecionados
    best_affinity_it.append(population_affinity[:10])

    population_select = population_affinity[:selection_size]

    population_clones = []
    for cell in population_select:
        cell_clones = clonalg.clone(cell, clone_rate)
        population_clones += cell_clones

    pop_clones_tmp = []
    for cell in population_clones:
        ind_tmp = clonalg.hypermutate(cell, mutation_rate, antigen, feature_min, feature_max)
        pop_clones_tmp.append(ind_tmp)
    population_clones = pop_clones_tmp
    del pop_clones_tmp

    population = clonalg.select(population_affinity, population_clones, population_size)

    population_rand = clonalg.create_random_cells(random_cells_num, feature_num, feature_min, feature_max)
    population_rand_affinity = [(cell, clonalg.affinity(cell, antigen)) for cell in population_rand]
    population_rand_affinity = sorted(population_rand_affinity, key=lambda x: abs(x[1]))

    population = clonalg.replace(population_affinity, population_rand_affinity, population_size)
    population = [cell[0] for cell in population]
    print('gen ', stop)
    stop += 1

gen  0
gen  1
gen  2
gen  3
gen  4
gen  5
gen  6
gen  7
gen  8
gen  9
gen  10
gen  11
gen  12
gen  13
gen  14
gen  15
gen  16
gen  17
gen  18
gen  19
gen  20
gen  21
gen  22
gen  23
gen  24
gen  25
gen  26
gen  27
gen  28
gen  29
gen  30
gen  31
gen  32
gen  33
gen  34
gen  35
gen  36
gen  37
gen  38
gen  39
gen  40
gen  41
gen  42
gen  43
gen  44
gen  45
gen  46
gen  47
gen  48
gen  49
gen  50
gen  51
gen  52
gen  53
gen  54
gen  55
gen  56
gen  57
gen  58
gen  59
gen  60
gen  61
gen  62
gen  63
gen  64
gen  65
gen  66
gen  67
gen  68
gen  69
gen  70
gen  71
gen  72
gen  73
gen  74
gen  75
gen  76
gen  77
gen  78
gen  79
gen  80
gen  81
gen  82
gen  83
gen  84
gen  85
gen  86
gen  87
gen  88
gen  89
gen  90
gen  91
gen  92
gen  93
gen  94
gen  95
gen  96
gen  97
gen  98
gen  99
gen  100
gen  101
gen  102
gen  103
gen  104
gen  105
gen  106
gen  107
gen  108
gen  109
gen  110
gen  111
gen  112
gen  113
gen  114
gen  115
gen  116
gen  117
gen  118
gen  119
gen  120
gen  121
gen  122
gen

In [227]:
bests_mean = []
iterations = [i for i in range(10000)]

for pop_it in best_affinity_it:
    bests_mean.append(np.mean([cell[1] for cell in pop_it]))

In [228]:
for i in range(len(pop_it)):
    population[i] = pop_it[i][0]

In [229]:
antibodies = pd.DataFrame(population)
antibodies = pd.DataFrame(scaler.inverse_transform(antibodies), columns=antibodies.columns)
antibodies = antibodies.head(10).to_numpy()

In [239]:
df_test = df.join(labels)
df_test = df_test.loc[df_test[4] == 2]

antigens = df_test.head(10)
antigens = antigens.to_numpy()

aff = []
aff_antigen = []
for i in range(len(antibodies)):
    for j in range(len(antibodies)):
         aff_antigen.append((abs(clonalg.affinity(antibodies[i], antigens[j]))))
    aff.append(min(aff_antigen))

In [240]:
for i in range(len(aff)):
    if aff[i] < threshold:
        print('The antibody ', i, ' has reacted to the antigen')

In [241]:
aff

[0.14484013582236632,
 0.14484013582236632,
 0.14484013582236632,
 0.14484013582236632,
 0.14484013582236632,
 0.14484013582236632,
 0.14484013582236632,
 0.14484013582236632,
 0.14484013582236632,
 0.10226836862965971]

In [242]:
antigens

array([[6.3, 3.3, 6. , 2.5, 2. ],
       [5.8, 2.7, 5.1, 1.9, 2. ],
       [7.1, 3. , 5.9, 2.1, 2. ],
       [6.3, 2.9, 5.6, 1.8, 2. ],
       [6.5, 3. , 5.8, 2.2, 2. ],
       [7.6, 3. , 6.6, 2.1, 2. ],
       [4.9, 2.5, 4.5, 1.7, 2. ],
       [7.3, 2.9, 6.3, 1.8, 2. ],
       [6.7, 2.5, 5.8, 1.8, 2. ],
       [7.2, 3.6, 6.1, 2.5, 2. ]])

In [243]:
antibodies

array([[7.66470984, 3.05302632, 7.6780216 , 0.04908238],
       [5.9545449 , 3.00747196, 5.96329604, 2.41871504],
       [7.19478893, 2.76410882, 7.9567203 , 0.78738739],
       [7.15345014, 3.6347247 , 1.26963782, 2.1824705 ],
       [6.74826373, 2.78713931, 3.76464726, 2.91053901],
       [7.49735946, 2.73141278, 6.30635155, 1.18474427],
       [6.00544414, 3.91770272, 3.75012747, 1.57058417],
       [7.73021627, 2.49932839, 5.89645769, 1.54370003],
       [6.844661  , 3.64765692, 4.19844545, 1.01230582],
       [7.38508064, 2.75005401, 7.72047226, 0.54666146]])

In [244]:
df_test = df.join(labels)
df_test = df_test.loc[df_test[4] == 1]

antigens = df_test.head(10)
antigens = antigens.to_numpy()

aff = []
aff_antigen = []
for i in range(len(antibodies)):
    for j in range(len(antibodies)):
        aff_antigen.append((abs(clonalg.affinity(antibodies[i], antigens[j]))))
    aff.append(min(aff_antigen))

aff

[2.044840135822365,
 0.9440279364919677,
 0.9440279364919677,
 0.05971682878503137,
 0.05971682878503137,
 0.05971682878503137,
 0.05971682878503137,
 0.05971682878503137,
 0.05971682878503137,
 0.05971682878503137]

In [245]:
antigens

array([[7. , 3.2, 4.7, 1.4, 1. ],
       [6.4, 3.2, 4.5, 1.5, 1. ],
       [6.9, 3.1, 4.9, 1.5, 1. ],
       [5.5, 2.3, 4. , 1.3, 1. ],
       [6.5, 2.8, 4.6, 1.5, 1. ],
       [5.7, 2.8, 4.5, 1.3, 1. ],
       [6.3, 3.3, 4.7, 1.6, 1. ],
       [4.9, 2.4, 3.3, 1. , 1. ],
       [6.6, 2.9, 4.6, 1.3, 1. ],
       [5.2, 2.7, 3.9, 1.4, 1. ]])

In [246]:
antibodies

array([[7.66470984, 3.05302632, 7.6780216 , 0.04908238],
       [5.9545449 , 3.00747196, 5.96329604, 2.41871504],
       [7.19478893, 2.76410882, 7.9567203 , 0.78738739],
       [7.15345014, 3.6347247 , 1.26963782, 2.1824705 ],
       [6.74826373, 2.78713931, 3.76464726, 2.91053901],
       [7.49735946, 2.73141278, 6.30635155, 1.18474427],
       [6.00544414, 3.91770272, 3.75012747, 1.57058417],
       [7.73021627, 2.49932839, 5.89645769, 1.54370003],
       [6.844661  , 3.64765692, 4.19844545, 1.01230582],
       [7.38508064, 2.75005401, 7.72047226, 0.54666146]])

In [247]:
df_test = df.join(labels)
df_test = df_test.loc[df_test[4] == 0]

antigens = df_test.head(10)
antigens = antigens.to_numpy()

aff = []
aff_antigen = []
for i in range(len(antibodies)):
    for j in range(len(antigens)):
        aff_antigen.append((abs(clonalg.affinity(antibodies[i], antigens[j]))))
    aff.append(min(aff_antigen))

aff

[7.044840135822366,
 5.944027936491969,
 5.944027936491969,
 2.8402831712149688,
 2.8402831712149688,
 2.8402831712149688,
 2.8402831712149688,
 2.8402831712149688,
 2.8402831712149688,
 2.8402831712149688]

In [248]:
antigens

array([[5.1, 3.5, 1.4, 0.2, 0. ],
       [4.9, 3. , 1.4, 0.2, 0. ],
       [4.7, 3.2, 1.3, 0.2, 0. ],
       [4.6, 3.1, 1.5, 0.2, 0. ],
       [5. , 3.6, 1.4, 0.2, 0. ],
       [5.4, 3.9, 1.7, 0.4, 0. ],
       [4.6, 3.4, 1.4, 0.3, 0. ],
       [5. , 3.4, 1.5, 0.2, 0. ],
       [4.4, 2.9, 1.4, 0.2, 0. ],
       [4.9, 3.1, 1.5, 0.1, 0. ]])

In [249]:
antibodies

array([[7.66470984, 3.05302632, 7.6780216 , 0.04908238],
       [5.9545449 , 3.00747196, 5.96329604, 2.41871504],
       [7.19478893, 2.76410882, 7.9567203 , 0.78738739],
       [7.15345014, 3.6347247 , 1.26963782, 2.1824705 ],
       [6.74826373, 2.78713931, 3.76464726, 2.91053901],
       [7.49735946, 2.73141278, 6.30635155, 1.18474427],
       [6.00544414, 3.91770272, 3.75012747, 1.57058417],
       [7.73021627, 2.49932839, 5.89645769, 1.54370003],
       [6.844661  , 3.64765692, 4.19844545, 1.01230582],
       [7.38508064, 2.75005401, 7.72047226, 0.54666146]])